In [16]:
import networkx as nx
import numpy as np
import pandas as pd
from pandas import DataFrame
import sys

In [17]:
top_pairs_path = "/Users/judithyemeli/Documents/CSE_6242/Project/MVR/network_graph/inputs_data/top_similarity_pairs.csv"
# all_pairs_path = "/Users/judithyemeli/Documents/CSE_6242/Project/MVR/network_graph/inputs_data/similarity_pairs.csv"
# high_sim_path = "/Users/judithyemeli/Documents/CSE_6242/Project/MVR/network_graph/inputs_data/high_sim.csv"

In [18]:
top_pairs_tableau = "/Users/judithyemeli/Documents/CSE_6242/Project/MVR/network_graph/inputs_data/top_pairs_tableau.csv"

In [19]:
df = pd.read_csv(top_pairs_path)
df.head()

,Source ID,Target ID,similarity,id_A,county_code_A,county_name_A,state_code_A,state_name_A,id_B,county_code_B,county_name_B,state_code_B,state_name_B,category
0,WV_005,WV_001,0.982010,WV_005,5,Boone County,54,West Virginia,WV_001,1,Barbour County,54,West Virginia,very similar
1,WV_001,WV_005,0.982010,WV_001,1,Barbour County,54,West Virginia,WV_005,5,Boone County,54,West Virginia,very similar
2,AR_003,MS_003,0.979938,AR_003,3,Ashley County,5,Arkansas,MS_003,3,Alcorn County,28,Mississippi,very similar
3,MS_003,AR_003,0.979938,MS_003,3,Alcorn County,28,Mississippi,AR_003,3,Ashley County,5,Arkansas,very similar
4,IA_009,IA_001,0.977880,IA_009,9,Audubon County,19,Iowa,IA_001,1,Adair County,19,Iowa,very similar


In [20]:
len(df)

1608

In [36]:
# df.rename(columns={'id_1': 'Source ID', 'id_2': 'Target ID'}, inplace=True)

In [21]:
# Read in Source File - NB this must match the schema requirements

df_InputData = df #
Src_Column = 'Source ID'
Tgt_Column = 'Target ID'

In [22]:
#Nodes are positioned using the Fruchterman-Reingold force-directed algorithm.

Q = nx.Graph()
arr_SrcTgt= np.array(df_InputData[[Src_Column, Tgt_Column]])
Q.add_edges_from(arr_SrcTgt)
dict_Coords = nx.spring_layout(Q)

df_Raw_Coords = DataFrame(dict_Coords)
df_Raw_Coords = df_Raw_Coords.T
df_Raw_Coords.columns = ['X','Y']
df_Raw_Coords['NodeName'] = df_Raw_Coords.index

#Add in a "Node Name" for cases where nodes do not link with another named node

df_Raw_Coords.fillna("Not Specified", inplace = True)

In [24]:
#Create bridge between main dataset and coordinates

arr_SrcTgt2 = arr_SrcTgt.reshape(1,(len(arr_SrcTgt)*2))
arr_SrcTgt2 = arr_SrcTgt2.reshape(-1)
df_SrcTgt = DataFrame(arr_SrcTgt2,columns=['NodeName'])
arr_Index = []
for i in range(1,(len(arr_SrcTgt)+1)):
        arr_Index.append(i)
        arr_Index.append(i)
df_SrcTgt['c_Index'] = arr_Index

#Join the datasets

df_InputData.index = df_InputData.index + 1

Merge_1 = pd.merge(
    left=df_SrcTgt,
    right=df_InputData,
    how="inner",
    left_on=df_SrcTgt['c_Index'],
    right_index=True,
)

df_MainDat = DataFrame(Merge_1)
df_MainDat = df_MainDat.drop(columns=['key_0'])

Merge_2 = pd.merge(
    left=df_Raw_Coords,
    right=df_MainDat,
    how="left",
    left_on=df_Raw_Coords['NodeName'],
    right_on=df_MainDat['NodeName'],
    suffixes=("", "_y"),
)

df_finaldat = DataFrame(Merge_2)
df_finaldat = df_finaldat.drop(columns=['key_0','NodeName_y'])

In [13]:
! pip install openpyxl

In [26]:
df_finaldat.to_excel("Network Data_high_sim_tableau.xlsx", sheet_name='Network')